In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Adrenocortical_Cancer/GSE19750'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Adrenocortical Carcinoma Gene Expression Profiling [Affymetrix]"
!Series_summary	"Background: Adrenocortical carcinoma (ACC) is associated with poor survival rates.  The objective of the study was to analyze ACC gene expression profiling data prognostic biomarkers and novel therapeutic targets."
!Series_summary	"Methods: 44 ACC and 4 normal adrenal glands were profiled on Affymetrix U133 Plus 2 expression microarrays and pathway and transcriptional enrichment analysis performed.  Protein levels were determined by western blot.  Drug efficacy was assessed against ACC cell lines.  Previously published expression datasets were analyzed as validation data sets."
!Series_summary	"Results: Pathway enrichment analysis identified marked dysregulation of cyclin-dependent kinases and mitosis.   Over-expression of PTTG1, which encodes securin, a negative regulator of p53, was identified as a marker of poor survival.  Median survival for patients with tumors 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0  # Adrenocortical_Cancer availability determined by the presence of 'Stage' field
age_row = 4  # 'age in years' field exists
gender_row = 3  # 'gender' field exists

def convert_trait(value):
    try:
        return 1 if value.split(': ')[1] != 'NA' and value.split(': ')[1] != 'Unknown' else None
    except IndexError:
        return None

def convert_age(value):
    try:
        age_str = value.split(': ')[1]
        return float(age_str) if age_str not in ['Unknown', 'NA'] else None
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    try:
        gender_val = value.split(': ')[1]
        return 1 if gender_val == 'M' else 0 if gender_val == 'F' else None
    except IndexError:
        return None

save_cohort_info('GSE19750', './preprocessed/Adrenocortical_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Adrenocortical_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Adrenocortical_Cancer/trait_data/GSE19750.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM493251': [None, None, None], 'GSM493252': [None, None, None], 'GSM493253': [None, None, None], 'GSM493254': [None, None, None], 'GSM493255': [1.0, 23.3, 1.0], 'GSM493256': [1.0, 56.5, 0.0], 'GSM493257': [1.0, 67.8, 1.0], 'GSM493258': [1.0, 72.1, 1.0], 'GSM493259': [1.0, 46.9, 1.0], 'GSM493260': [1.0, 26.7, 0.0], 'GSM493261': [1.0, 48.5, 1.0], 'GSM493262': [1.0, 36.9, 0.0], 'GSM493263': [1, None, 0], 'GSM493264': [1.0, 53.2, 1.0], 'GSM493265': [1.0, 37.0, 0.0], 'GSM493266': [1.0, 54.2, 0.0], 'GSM493267': [1.0, 67.3, 0.0], 'GSM493268': [1.0, 27.7, 0.0], 'GSM493269': [1, None, 1], 'GSM493270': [1.0, 58.0, 0.0], 'GSM493271': [1.0, 56.7, 0.0], 'GSM493272': [1.0, 42.0, 1.0], 'GSM493273': [nan, 46.0, 0.0], 'GSM1094056': [nan, 20.0, 0.0], 'GSM1094057': [1.0, 68.0, 0.0], 'GSM1094058': [1.0, 45.0, 0.0], 'GSM1094059': [1.0, 46.0, 1.0], 'GSM1094060': [1.0, 32.0, 0.0], 'GSM1094061': [1.0, 43.0, 1.0], 'GSM1094062': [1.0, 45.0, 0.0], 'GSM1094063': [nan, 40.0, 1.0], 'GSM1094064': [nan, 52.0, 1.0]

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the identifier and gene symbol keys
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Adrenocortical_Cancer/gene_data/GSE19750.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Adrenocortical_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE19750', './preprocessed/Adrenocortical_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Adrenocortical_Cancer/GSE19750.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Adrenocortical_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Adrenocortical_Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 43.5
  50% (Median): 53.1
  75%: 58.75
Min: 23.3
Max: 72.1
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 38.24% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

